## Import Lib

In [1]:
# Import
import pandas as pd
import numpy as np
import os
import random
import pyarrow.dataset as ds
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import optuna
from tqdm import tqdm
import gc
import warnings
warnings.filterwarnings('ignore')

## Config

In [2]:
# Enhanced Settings
CFG = {
    'BATCH_SIZE': 800_000,
    'NUM_BOOST_ROUND': 2000,
    'LEARNING_RATE': 0.05,
    'SEED': 42,
    'TEST_SIZE': 0.2,
    'EARLY_STOPPING': 100,
    'N_FOLDS': 5,
    'OPTUNA_TRIALS': 20
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED'])

## Feature Engineering

In [3]:
def create_interaction_features(df, cat_cols):
    interaction_features = []
    for i, col1 in enumerate(cat_cols):
        for col2 in cat_cols[i+1:]:
            if col1 in df.columns and col2 in df.columns:
                new_col = f"{col1}_{col2}_interaction"
                df[new_col] = df[col1].astype(str) + "_" + df[col2].astype(str)
                interaction_features.append(new_col)
    le = LabelEncoder()
    for col in interaction_features:
        df[col] = le.fit_transform(df[col].astype(str))
    return df, interaction_features

In [4]:
import pandas as pd
import numpy as np

def create_derived_features(df):
    """
    history_a, history_b, 정보영역(feat_a~e) 기반 파생 변수 생성
    gender, age_group 상호작용(history 파생 변수와 곱)
    """
    # gender, age_group을 숫자형으로 변환 (문자열일 경우 대비)
    if 'gender' in df.columns:
        df['gender'] = pd.to_numeric(df['gender'], errors='coerce').fillna(0).astype(int)
    if 'age_group' in df.columns:
        df['age_group'] = pd.to_numeric(df['age_group'], errors='coerce').fillna(0).astype(int)
    
    # 1) history_a, history_b 통계
    history_a_cols = [c for c in df.columns if c.startswith("history_a_")]
    history_b_cols = [c for c in df.columns if c.startswith("history_b_")]
    
    if history_a_cols:
        df['history_a_sum'] = df[history_a_cols].sum(axis=1)
        df['history_a_mean'] = df[history_a_cols].mean(axis=1)
        df['history_a_max'] = df[history_a_cols].max(axis=1)
        df['history_a_min'] = df[history_a_cols].min(axis=1)
    
    if history_b_cols:
        df['history_b_sum'] = df[history_b_cols].sum(axis=1)
        df['history_b_mean'] = df[history_b_cols].mean(axis=1)
        df['history_b_max'] = df[history_b_cols].max(axis=1)
        df['history_b_min'] = df[history_b_cols].min(axis=1)
    
    # 2) gender * history, age_group * history 상호작용 변수
    if 'gender' in df.columns:
        df['gender_history_a_sum'] = df['gender'] * df['history_a_sum']
        df['gender_history_a_mean'] = df['gender'] * df['history_a_mean']
        df['gender_history_b_sum'] = df['gender'] * df['history_b_sum']
        df['gender_history_b_mean'] = df['gender'] * df['history_b_mean']
    
    if 'age_group' in df.columns:
        df['age_history_a_sum'] = df['age_group'] * df['history_a_sum']
        df['age_history_a_mean'] = df['age_group'] * df['history_a_mean']
        df['age_history_b_sum'] = df['age_group'] * df['history_b_sum']
        df['age_history_b_mean'] = df['age_group'] * df['history_b_mean']
    
    return df

In [5]:
def create_time_features(df):
    if 'hour' in df.columns:
        try:
            df['hour'] = pd.to_numeric(df['hour'], errors='coerce').fillna(0).astype(int)
            df['is_morning'] = ((df['hour'] >=6) & (df['hour']<12)).astype(int)
            df['is_afternoon'] = ((df['hour']>=12)&(df['hour']<18)).astype(int)
            df['is_evening'] = ((df['hour']>=18)&(df['hour']<24)).astype(int)
            df['is_night'] = ((df['hour']>=0)&(df['hour']<6)).astype(int)
            df['is_peak_hour'] = df['hour'].isin([9,10,11,14,15,16,19,20,21]).astype(int)
            df['hour_sin'] = np.sin(2*np.pi*df['hour']/24)
            df['hour_cos'] = np.cos(2*np.pi*df['hour']/24)
        except:
            pass
    if 'day_of_week' in df.columns:
        try:
            df['day_of_week'] = pd.to_numeric(df['day_of_week'], errors='coerce').fillna(0).astype(int)
            df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
            df['is_monday'] = (df['day_of_week']==0).astype(int)
            df['is_friday'] = (df['day_of_week']==4).astype(int)
            df['dow_sin'] = np.sin(2*np.pi*df['day_of_week']/7)
            df['dow_cos'] = np.cos(2*np.pi*df['day_of_week']/7)
        except:
            pass
    return df

In [ ]:
def preprocess_features(df, feature_cols, seq_col, is_train=True, target_encoders=None):
    cat_cols = ["gender","age_group","inventory_id","day_of_week","hour"]
    df, interaction_features = create_interaction_features(df, cat_cols)
    df = create_time_features(df)
    del df['seq']
    if is_train and 'clicked' in df.columns:
        for col in cat_cols:
            if col in df.columns:
                try:
                    target_mean = df.groupby(col)['clicked'].mean()
                    df[f'{col}_target_mean'] = df[col].map(target_mean).fillna(df['clicked'].mean())
                except:
                    pass
    elif target_encoders is not None:
        for col, encoder_dict in target_encoders.items():
            if col in df.columns and encoder_dict:
                df[f'{col}_target_mean'] = df[col].map(encoder_dict).fillna(0.5)
    new_feature_cols = [col for col in df.columns if col not in ['clicked','ID']]
    return df, new_feature_cols

def encode_categorical_features(df, cat_cols):
    for col in cat_cols:
        if col in df.columns:
            try:
                freq_encoding = df[col].value_counts().to_dict()
                df[f'{col}_freq'] = df[col].map(freq_encoding).fillna(0)
                df[col] = df[col].astype('category').cat.codes
            except:
                df[col] = 0
                df[f'{col}_freq'] = 0
    return df

## Optuna

In [7]:
def optimize_xgb_params(X_train, y_train, X_val, y_val):
    def objective(trial):
        params = {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "booster": "gbtree",
            "tree_method": "hist",
            "learning_rate": trial.suggest_float("learning_rate",0.01,0.2),
            "max_depth": trial.suggest_int("max_depth",3,12),
            "min_child_weight": trial.suggest_int("min_child_weight",1,10),
            "subsample": trial.suggest_float("subsample",0.4,1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree",0.4,1.0),
            "gamma": trial.suggest_float("gamma",0.0,5.0),
            "lambda": trial.suggest_float("lambda",0.0,5.0),
            "alpha": trial.suggest_float("alpha",0.0,5.0),
            "seed": CFG['SEED']
        }
        dtrain = xgb.DMatrix(X_train,label=y_train)
        dval = xgb.DMatrix(X_val,label=y_val)
        bst = xgb.train(
            params,dtrain,num_boost_round=1000,
            evals=[(dval,"val")],
            early_stopping_rounds=50,
            verbose_eval=False
        )
        preds = bst.predict(dval)
        auc = roc_auc_score(y_val,preds)
        return auc
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=CFG['SEED']))
    study.optimize(objective,n_trials=CFG['OPTUNA_TRIALS'], show_progress_bar=True)
    return study.best_params

## Train

In [8]:
def downsample_data(df):
    clicked_1 = df[df['clicked']==1]
    clicked_0 = df[df['clicked']==0]
    if len(clicked_0) > len(clicked_1)*3:
        clicked_0 = clicked_0.sample(n=len(clicked_1)*3, random_state=CFG['SEED'])
    return pd.concat([clicked_1,clicked_0],axis=0).sample(frac=1,random_state=CFG['SEED']).reset_index(drop=True)

In [9]:
def train_xgb_with_cv(data_path):
    dataset = ds.dataset(data_path, format="parquet")
    models=[]
    feature_names=None
    target_encoders={}
    batch_num=0
    best_params=None
    for batch in dataset.to_batches(batch_size=CFG['BATCH_SIZE']):
        batch_num+=1
        df = batch.to_pandas()
        df = downsample_data(df)
        feature_cols = [c for c in df.columns if c not in {'clicked','seq','ID'}]
        df, feature_cols = preprocess_features(df, feature_cols,'seq',is_train=True)
        if batch_num==1:
            cat_cols = ["gender","age_group","inventory_id","day_of_week","hour"]
            for col in cat_cols:
                if col in df.columns and 'clicked' in df.columns:
                    try:
                        target_encoders[col] = df.groupby(col)['clicked'].mean().to_dict()
                    except:
                        target_encoders[col] = {}
        df = encode_categorical_features(df, ["gender","age_group","inventory_id","day_of_week","hour"])
        available_features = [col for col in feature_cols if col in df.columns]
        if feature_names is None:
            feature_names = available_features.copy()
        X = df[available_features].fillna(0)
        y = df['clicked']
        if batch_num==1:
            sample_size = min(50000,len(X))
            sample_indices = np.random.choice(len(X), sample_size, replace=False)
            X_sample = X.iloc[sample_indices]
            y_sample = y.iloc[sample_indices]
            X_temp,X_val_temp,y_temp,y_val_temp = train_test_split(
                X_sample,y_sample,test_size=0.2,random_state=CFG['SEED'], stratify=y_sample
            )
            best_params = optimize_xgb_params(X_temp,y_temp,X_val_temp,y_val_temp)
            del X_temp,X_val_temp,y_temp,y_val_temp,X_sample,y_sample
        skf = StratifiedKFold(n_splits=CFG['N_FOLDS'], shuffle=True, random_state=CFG['SEED'])
        fold_aucs=[]
        for fold,(train_idx,val_idx) in enumerate(skf.split(X,y)):
            X_train,X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train,y_val = y.iloc[train_idx], y.iloc[val_idx]
            dtrain = xgb.DMatrix(X_train,label=y_train)
            dval = xgb.DMatrix(X_val,label=y_val)
            model = xgb.train(
                best_params or {
                    "objective":"binary:logistic",
                    "eval_metric":"auc",
                    "learning_rate":CFG['LEARNING_RATE'],
                    "max_depth":6,
                    "subsample":0.8,
                    "colsample_bytree":0.8,
                    "seed":CFG['SEED']
                },
                dtrain,
                num_boost_round=CFG['NUM_BOOST_ROUND'],
                evals=[(dtrain,"train"),(dval,"valid")],
                early_stopping_rounds=CFG['EARLY_STOPPING'],
                verbose_eval=100
            )
            val_pred = model.predict(dval)
            fold_auc = roc_auc_score(y_val,val_pred)
            fold_aucs.append(fold_auc)
            models.append(model)
        print(f"Batch {batch_num} Average CV AUC: {np.mean(fold_aucs):.4f} (+/- {np.std(fold_aucs):.4f})")
        del df,X,y
        gc.collect()
    return models, feature_names, target_encoders

models, feature_names, target_encoders = train_xgb_with_cv("./data/train.parquet")
print("=== High Performance XGBoost CTR Prediction ===")

[I 2025-09-28 01:24:42,541] A new study created in memory with name: no-name-70e09348-e5f4-47ee-861c-820648181497


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-09-28 01:24:44,554] Trial 0 finished with value: 0.7255946550270277 and parameters: {'learning_rate': 0.08116262258099886, 'max_depth': 12, 'min_child_weight': 8, 'subsample': 0.759195090518222, 'colsample_bytree': 0.4936111842654619, 'gamma': 0.7799726016810132, 'lambda': 0.2904180608409973, 'alpha': 4.330880728874676}. Best is trial 0 with value: 0.7255946550270277.
[I 2025-09-28 01:24:46,079] Trial 1 finished with value: 0.7191399568178086 and parameters: {'learning_rate': 0.12421185223120967, 'max_depth': 10, 'min_child_weight': 1, 'subsample': 0.9819459112971965, 'colsample_bytree': 0.899465584480253, 'gamma': 1.0616955533913808, 'lambda': 0.9091248360355031, 'alpha': 0.9170225492671691}. Best is trial 0 with value: 0.7255946550270277.
[I 2025-09-28 01:24:47,689] Trial 2 finished with value: 0.727029615179241 and parameters: {'learning_rate': 0.06780602616231217, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.5747374841188252, 'colsample_bytree': 0.7671117368334277,

## Predict

In [10]:
def predict_test_data(models,test_path,feature_names,target_encoders):
    test_df = pd.read_parquet(test_path, engine="pyarrow")
    if 'ID' in test_df.columns:
        test_ids = test_df['ID'].copy()
        test_df = test_df.drop(columns=['ID'])
    feature_cols = [c for c in test_df.columns if c!='seq']
    test_df, feature_cols = preprocess_features(test_df, feature_cols,'seq',is_train=False,target_encoders=target_encoders)
    test_df = encode_categorical_features(test_df, ["gender","age_group","inventory_id","day_of_week","hour"])
    for feature in feature_names:
        if feature not in test_df.columns:
            test_df[feature]=0
    test_features = test_df[feature_names].fillna(0)
    predictions = np.zeros(len(test_df))
    for model in models:
        dtest = xgb.DMatrix(test_features)
        predictions += model.predict(dtest)
    predictions /= len(models)
    return predictions

test_predictions = predict_test_data(models,"./data/test.parquet",feature_names,target_encoders)

## Submission

In [11]:
submission = pd.read_csv('./sample_submission.csv')
submission['clicked'] = test_predictions
submission.to_csv('./high_performance_xgb_submit_v2.csv', index=False)